In [ ]:
import pandas as pd
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score


In [ ]:
df = pd.read_csv('../data/customer_support_tickets_cleaned.csv')

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_cleaned'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

print(f'Training set size: {X_train.shape[0]}')
print(f'Test set size: {X_test.shape[0]}')


In [ ]:
dummy = DummyClassifier(strategy='most_frequent') # Using most frequent class strategy by initialization Majority Classifier

dummy.fit(X_train, y_train) # fit the model

y_pred = dummy.predict(X_test) # make predictions

print("Dummy Classifier Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred)) 

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english') # Create TF-IDF vectorizer to vectorize text data
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression(max_iter=1000) # Initialize Logistic Regression model training
lr.fit(X_train_vec, y_train) # Fit the model to the training data

y_pred_lr = lr.predict(X_test_vec) # Make predictions on the test data

acc_lr = accuracy_score(y_test, y_pred_lr)
f1_lr = f1_score(y_test, y_pred_lr, average='weighted')
cm_lr = confusion_matrix(y_test, y_pred_lr)

print("TF-IDF + vectorizer + Logistic Regression model trained.")
print("Logistic Regression Performance:")
print(f"Accuracy: {acc_lr}")
print(f"F1 Score: {f1_lr}")
print(f"Confusion Matrix:\n {cm_lr}")

print(f"Classification Report:\n {classification_report(y_test, y_pred_lr)}")



In [ ]:
baseline_results = pd.DataFrame({
    'Model': ['Dummy Classifier', 'Logistic Regression'],
    'Accuracy': [accuracy_score(y_test, y_pred), accuracy_score(y_test, y_pred_lr)],
    'F1 Score': [f1_score(y_test, y_pred, average='weighted'), f1_score(y_test, y_pred_lr, average='weighted')]
})
print("\nBaseline Model Comparison:")
print(baseline_results)

In [ ]:
#saving metrics to json file
#preparing metrics dictionary
metrics = {
    'Dummy Classifier': {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred, average='weighted'),
        'confusion_matrix': confusion_matrix(y_test, y_pred).tolist()
    },
    'tfidf + logreg': {
        'Accuracy': acc_lr,
        'F1 Score': f1_lr,
        'confusion_matrix': cm_lr.tolist()
    }
}

with open('../src/baseline_model_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

    print("Baseline model metrics saved to '../src/baseline_model_metrics.json'")


In [ ]:
print("Classification Report for Logistic Regression:")
print(classification_report(y_test, y_pred_lr)) 